In [5]:
!pip install en_core_web_sm 

ERROR: Could not find a version that satisfies the requirement en_core_web_sm (from versions: none)
ERROR: No matching distribution found for en_core_web_sm


In [1]:
import numpy as np
import json
import re
import tensorflow as tf
import random
import spacy
nlp = spacy.load('en_core_web_sm')

ConnectTimeout: HTTPSConnectionPool(host='raw.githubusercontent.com', port=443): Max retries exceeded with url: /explosion/spacy-models/master/compatibility.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002BD34FFA750>, 'Connection to raw.githubusercontent.com timed out. (connect timeout=None)'))

In [2]:
with open('intents.json') as f:
    intents = json.load(f)

In [3]:
def preprocessing(line):
    line = re.sub(r'[^a-zA-z.?!\']', ' ', line)
    line = re.sub(r'[ ]+', ' ', line)
    return line

In [4]:
inputs, targets = [], []
classes = []
intent_doc = {}

for intent in intents['intents']:
    if intent['intent'] not in classes:
        classes.append(intent['intent'])
    if intent['intent'] not in intent_doc:
        intent_doc[intent['intent']] = []
        
    for text in intent['text']:
        inputs.append(preprocessing(text))
        targets.append(intent['intent'])
        
    for response in intent['responses']:
        intent_doc[intent['intent']].append(response)

In [5]:
def tokenize_data(input_list):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<unk>')
    tokenizer.fit_on_texts(input_list)
    input_seq = tokenizer.texts_to_sequences(input_list)
    input_seq = tf.keras.preprocessing.sequence.pad_sequences(input_seq, padding='pre')
    return tokenizer, input_seq

# preprocess input data
tokenizer, input_tensor = tokenize_data(inputs)

In [6]:
def create_categorical_target(targets):
    word={}
    categorical_target=[]
    counter=0
    for trg in targets:
        if trg not in word:
            word[trg]=counter
            counter+=1
        categorical_target.append(word[trg])
    
    categorical_tensor = tf.keras.utils.to_categorical(categorical_target, num_classes=len(word), dtype='int32')
    return categorical_tensor, dict((v,k) for k, v in word.items())

# preprocess output data
target_tensor, trg_index_word = create_categorical_target(targets)

In [7]:
# hyperparameters
epochs=50
vocab_size=len(tokenizer.word_index) + 1
embed_dim=512
units=128
target_length=target_tensor.shape[1]

In [8]:
# build RNN Model with tensorflow
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embed_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units, dropout=0.2)),
    tf.keras.layers.Dense(units, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(target_length, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 512)         66048     
                                                                 
 bidirectional (Bidirection  (None, 256)               656384    
 al)                                                             
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 22)                2838      
                                                                 
Total params: 758166 (2.89 MB)
Trainable params: 758166 (2.89 MB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [9]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)

# train the model
model.fit(input_tensor, target_tensor, epochs=epochs, callbacks=[early_stop])

Epoch 1/50
5/5 [==============================] - 4s 46ms/step - loss: 3.1083 - accuracy: 0.0420
Epoch 2/50
5/5 [==============================] - 0s 45ms/step - loss: 2.6635 - accuracy: 0.3007
Epoch 3/50
5/5 [==============================] - 0s 47ms/step - loss: 1.7966 - accuracy: 0.4406
Epoch 4/50
5/5 [==============================] - 0s 47ms/step - loss: 0.9840 - accuracy: 0.6503
Epoch 5/50
5/5 [==============================] - 0s 48ms/step - loss: 0.7787 - accuracy: 0.7483
Epoch 6/50
5/5 [==============================] - 0s 47ms/step - loss: 0.4290 - accuracy: 0.8741
Epoch 7/50
5/5 [==============================] - 0s 47ms/step - loss: 0.1433 - accuracy: 0.9790
Epoch 8/50
5/5 [==============================] - 0s 47ms/step - loss: 0.1600 - accuracy: 0.9371
Epoch 9/50
5/5 [==============================] - 0s 47ms/step - loss: 0.1362 - accuracy: 0.9650
Epoch 10/50
5/5 [==============================] - 0s 49ms/step - loss: 0.1217 - accuracy: 0.9790
Epoch 11/50
5/5 [============

In [10]:
# Save the trained model
model.save('chatbot_model.keras')